In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [37]:
D = 1.7

In [38]:
def icc2PL(a, b, theta):    
    prob = 1/(1+np.exp(-D*a*(theta-b)))
    return prob

In [39]:
A = (1.0, 1.5, 0.7, 1.2)
B = (1.5, 0.5, 0.0, -1.0)
ir = [
    [0,0,1,1],
    [0,1,0,1],
    [1,0,1,1],
    [1,1,1,1],
]
U = ir[2]

In [40]:
t0 = np.log(sum(U)/(len(U)-sum(U)))

In [41]:
t0

1.0986122886681098

In [42]:
for a,b in zip(A,B):
    print (icc2PL(a,b,t0))

0.3357349774023402
0.8214879753649259
0.7870691464401657
0.9863620760816233


In [43]:
def LL(U,A,B,theta): # 対数尤度関数
    sm = 0
    for u,a,b in zip(U,A,B):
        sm += (u*np.log(icc2PL(a,b,theta))+(1-u)*np.log(1-icc2PL(a,b,theta)))
    return sm

In [44]:
LL(u,A,B,t0)

-3.067703451414232

In [45]:
def dLL(U,A,B,theta): # LLの一次導関数
    sm = 0
    for u,a,b in zip(U,A,B):
        sm += D*a*(u-icc2PL(a,b,theta))
    return sm    

In [47]:
dLL(U,A,B,t0)

-0.6843347182348481

In [48]:
def ddLL(A,B,theta):
    sm = 0
    for a,b in zip(A,B):
        sm += -D**2 * (a**2 * icc2PL(a,b,theta)*(1-icc2PL(a,b,theta)))
    return sm    

In [49]:
ddLL(A,B,t0)

-1.8913889846008611

In [52]:
e = 1e-4
conv = 0
while(conv==0):
    t1 = t0 - dLL(U,A,B,t0)/ddLL(A,B,t0)
    if (abs(t1-t0)<e):
        conv = 1
    t0 = t1

In [53]:
print(t1)

0.776437984268933


In [56]:
LL(U,A,B,t1)

-2.9535494010580607

In [62]:
def MLE(ir, A, B):
    ans = []
    for U in ir:
        print(U)
        try: # 最尤推定では全問正解/不正解の場合に能力推定値を正しく求められない
            t0 = np.log(sum(U)/(len(U)-sum(U)))
        except:
            ans.append(np.nan)
            continue
        e = 1e-4
        conv = 0
        while(conv==0):
            t1 = t0 - dLL(U,A,B,t0)/ddLL(A,B,t0)
            if (abs(t1-t0)<e):
                conv = 1
            t0 = t1
        ans.append(t1)
    return ans

In [63]:
MLE(ir, A, B)

[0, 0, 1, 1]
[0, 1, 0, 1]
[1, 0, 1, 1]
[1, 1, 1, 1]


[0.07044249667892975, 0.6351831574812188, 0.776437984268933, nan]

In [64]:
def LG(U,A,B,theta,mu,sigma): # 対数事後分布
    return LL(U,A,B,theta) - 1/2*((theta-mu)/sigma)**2

In [65]:
def dLG(U,A,B,theta,mu,sigma): # 一次導関数
    return dLL(U,A,B,theta) - (theta-mu)/sigma**2

In [72]:
def ddLG(A,B,theta,sigma): # 二次導関数
    return ddLL(A,B,theta) - 1/sigma**2

In [73]:
def MAP(ir, A, B):
    e = 1e-4
    mu = 0
    sigma = 1
    ans = []
    for U in ir:
        print(U)
        try: # 全問正解/不正解の場合はこの初期化は使えない
            t0 = np.log(sum(U)/(len(U)-sum(U)))
        except:
            ans.append(np.nan)
            continue
        
        conv = 0
        while(conv==0):
            t1 = t0 - dLG(U,A,B,t0,mu,sigma)/ddLG(A,B,t0,sigma)
            if (abs(t1-t0)<e):
                conv = 1
            t0 = t1
        ans.append(t1)
    return ans

In [74]:
MAP(ir, A, B)

[0, 0, 1, 1]
[0, 1, 0, 1]
[1, 0, 1, 1]
[1, 1, 1, 1]


[0.04811088923817803, 0.4534248452256026, 0.5504642571004835, nan]

In [88]:
import numpy as np
from scipy.stats import norm
Xm = np.linspace(-4,4,15)

Wm = norm.pdf(Xm,0,1)
Wm = Wm/sum(Wm)

In [90]:
np.round(Xm,4)

array([-4.    , -3.4286, -2.8571, -2.2857, -1.7143, -1.1429, -0.5714,
        0.    ,  0.5714,  1.1429,  1.7143,  2.2857,  2.8571,  3.4286,
        4.    ])

In [89]:
np.round(Wm,4)

array([1.000e-04, 6.000e-04, 3.800e-03, 1.670e-02, 5.240e-02, 1.186e-01,
       1.936e-01, 2.280e-01, 1.936e-01, 1.186e-01, 5.240e-02, 1.670e-02,
       3.800e-03, 6.000e-04, 1.000e-04])

In [110]:
def L(U,A,B,theta):
    prod = 1
    for u,a,b in zip(U,A,B):
        prod *= (icc2PL(a,b,theta)**u * (1-icc2PL(a,b,theta))**(1-u))
    return prod

In [111]:
Xm[9]

1.1428571428571423

In [128]:
for U in ir:
    print(U)
    Lm = []
    for xm in Xm:
        lm =  L(U,A,B,xm)
        Lm.append(lm)
        #print(np.round(lm,4))
    Lm = np.array(Lm)
    Gm = Lm * Wm / sum(Lm*Wm)
    eap_theta = sum(Xm*Gm)
    print(np.round(eap_theta,7))

[0, 0, 1, 1]
0.0093134
[0, 1, 0, 1]
0.4515764
[1, 0, 1, 1]
0.5612238
[1, 1, 1, 1]
1.4769317


In [166]:
# 8.2.2

In [167]:
import pandas as pd

In [168]:
data_EM = pd.DataFrame(
    [
        [1,1,1,1,18],
        [1,1,1,0,1],
        [1,1,0,1,7],
        [1,1,0,0,1],
        [1,0,1,1,4],
        [1,0,1,0,1],
        [1,0,0,1,4],
        [1,0,0,0,1],
        [0,1,1,1,31],
        [0,1,1,0,1],
        [0,1,0,1,17],
        [0,1,0,0,1],
        [0,0,1,1,35],
        [0,0,1,0,6],
        [0,0,0,1,42],
        [0,0,0,0,30],
    ],
    columns = ['1','2','3','4','人数']
)